In [1]:
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection  import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
import LightGBM as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder

In [2]:
## 读取text_feature 文件
text_features = pd.read_csv('./temp/text_features.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (97,98,99,100,106,124,134,135,137,140,141,142,143,144,150,171,172,175,176,177,178,179,187,189,190,191,192,193,194,195,196,197,198,200,201,202,203,204,205,206,207,208,209,210) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
##眼科涉及到的列
eye_col = ['1302', '1303' ,'1304', '1305', '1313', '1314', '1315' ,'1316','1328','1329','1330']


In [7]:
## 对挑选出来的列 做one-hot
def oneHot(tmp):
    vid = tmp['vid']
    tmp = tmp.drop(['vid'],axis=1)
    enc=OneHotEncoder()
    enc.fit(tmp)
    data=enc.transform(tmp).toarray()
    ans = pd.DataFrame(index=vid,data=data)
    return ans

In [11]:
def clean_eyes(df,cols):
    
    def func_eye(x):
        x = str(x)
        if x == '正常'or x =='未见异常'or x== '活动自如' or x=='未见明显异常':
            return 1
        elif x == '眼底检查未发现明显异常' or x == '眼底检查未见异常' or x == '眼底检查(眼底)未发现明显异常':
            return 1
        elif x == 'nan' or x=='无' or ('nan' in x):
            return 0
        else:
            return 2
    
    res = pd.DataFrame(data=df['vid'])
    for col in cols:
        res[col] = df[col].apply(func_eye)
    
    ans = oneHot(res)
    return ans

In [12]:
## 生成 眼科的 one-hot 向量
eye_col_csv = clean_eyes(text_features,eye_col)

In [15]:
### 脑科 
brain = ['1402']

In [16]:
def clean_brain(df,cols):
    
    def brain_func(x):
        x = str(x)
        if '正常' in x  or '未见明显异常' in x:
            return 1
        elif x == 'nan':
            return 0
        else:
            return 2
    
    
    res = pd.DataFrame(data=df['vid'])
    
    for col in cols:
        res[col] = df[col].apply(brain_func)
        
    ans = oneHot(res)
    return ans

In [17]:
brain_col_csv = clean_brain(text_features,brain)

In [20]:
##阴性 阳性 健康疾病 
health = ['2228','2229','2230','2231','2233','2302']

In [21]:
def clean_health(df,cols):
    
    def health_func(x):
        x = str(x)
        if '正常' in x  or '未见明显异常' in x:
            return 1
        elif x == 'nan':
            return 0
        else:
            return 2
    
    
    res = pd.DataFrame(data=df['vid'])
    
    for col in cols:
        res[col] = df[col].apply(health_func)
        
    ans = oneHot(res)
    return ans

In [22]:
health_col_csv = clean_health(text_features,health)

In [23]:
## 妇科检测
female = ['2501']

In [24]:
def clean_female(df,cols):
    
    def female_func(x):
        x = str(x)
        if '未见' in x or '正常' in x:
            return 1
        elif 'nan' == x:
            return 0
        else:
            return 2
    
    
    res = pd.DataFrame(data=df['vid'])
    
    for col in cols:
        res[col] = df[col].apply(female_func)
        
    ans = oneHot(res)
    return ans

In [25]:
female_col_csv = clean_female(text_features,female)

In [26]:
## + - 阴 阳
plus_minus = ['300018','300019','3189','3190','3191','3192','3194','3195','3196','3197']

In [27]:
## 根据上面列的信息，一共分为5类
def clean_plus_minus(df,cols):
    
    def plus_minus_func(x):
        x = str(x)
        if '阴' in x or x == '正常':
            return 1
        elif x == '-':
            return 1
        elif '阳' in x:
            return 1
        elif x == '+-':
            return 2
        elif x == '++':
            return 3
        elif x == '+++':
            return 4
        elif 'nan' == x:
            return 0
        else:
            return 5
    
    res = pd.DataFrame(data=df['vid'])
    
    for col in cols:
        res[col] = df[col].apply(plus_minus_func)
        
    ans = oneHot(res)
    return ans

In [28]:
plus_minus_csv = clean_plus_minus(text_features,plus_minus)

In [29]:
## 骨质酥松
bone = ['3601']

In [32]:
def clean_bone(df,cols):
    
    def bone_func(x):
        x = str(x)

        if '正常' in x:
            return 1
        elif  'nan' == x:
            return 0
        else:
            return 2
    
    res = pd.DataFrame(data=df['vid'])
    
    for col in cols:
        res[col] = df[col].apply(bone_func)
        
    ans = oneHot(res)
    return ans

In [33]:
bone_col_csv = clean_bone(text_features,bone)

In [34]:
#血液检测
blood = ['4001']

In [35]:
def clean_blood(df,cols):
    
    def blood_func(x):
        x = str(x)

        if '正常' in x:
            return 1
        elif  'nan' == x:
            return 0
        else:
            return 2
    
    res = pd.DataFrame(data=df['vid'])
    
    for col in cols:
        res[col] = df[col].apply(blood_func)
        
    ans = oneHot(res)
    return ans

In [36]:
blood_col_csv = clean_blood(text_features,blood)

In [37]:
## 颜色 （黄色，淡黄） 这一列的处理
color = ['3399']

In [38]:
def clean_color(df,cols):
    
    def color_func(x):
        x = str(x)

        if '正常' in x:
            return 1
        elif  'nan' == x:
            return 0
        else:
            return 2
    
    res = pd.DataFrame(data=df['vid'])
    
    for col in cols:
        res[col] = df[col].apply(color_func)
        
    ans = oneHot(res)
    return ans

In [39]:
color_col_csv = clean_color(text_features,color)

In [40]:
###幽门螺旋杆菌
hp = ['3301']

In [41]:
def clean_hp(df,cols):
    
    def hp_func(x):
        x = str(x)

        if '正常' in x:
            return 1
        elif  'nan' == x:
            return 0
        else:
            return 2
    
    res = pd.DataFrame(data=df['vid'])
    
    for col in cols:
        res[col] = df[col].apply(hp_func)
        
    ans = oneHot(res)
    return ans

In [42]:
hp_col_csv = clean_hp(text_features,hp)

In [43]:
one_hot_all = pd.concat([eye_col_csv,brain_col_csv],axis=1,join='inner')
one_hot_all = pd.concat([one_hot_all,health_col_csv],axis=1,join='inner')
one_hot_all = pd.concat([one_hot_all,female_col_csv],axis=1,join='inner')
one_hot_all = pd.concat([one_hot_all,plus_minus_csv],axis=1,join='inner')
one_hot_all = pd.concat([one_hot_all,bone_col_csv],axis=1,join='inner')
one_hot_all = pd.concat([one_hot_all,blood_col_csv],axis=1,join='inner')
one_hot_all = pd.concat([one_hot_all,color_col_csv],axis=1,join='inner')
one_hot_all = pd.concat([one_hot_all,hp_col_csv],axis=1,join='inner')

In [47]:
#57298 rows × 116 columns
one_hot_all.to_csv('./temp/one_hot_all.csv')